<a href="https://colab.research.google.com/github/OneFineStarstuff/OneFineStarstuff/blob/main/Deployment_Optimizations_Quantization%2C_Pruning%2C_and_ONNX_Conversion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install onnx

In [ ]:
import torch
import torch.nn as nn
import torch.onnx
from transformers import AutoModelForCausalLM, AutoTokenizer

# Initialize student model and tokenizer
student_model = AutoModelForCausalLM.from_pretrained("distilgpt2")

# Create a dummy input for the export
tokenizer = AutoTokenizer.from_pretrained("distilgpt2")
dummy_input = tokenizer("This is a dummy input", return_tensors="pt")

# Create a wrapper model to handle input_ids only
class WrappedModel(nn.Module):
    def __init__(self, model):
        super(WrappedModel, self).__init__()
        self.model = model

    def forward(self, input_ids):
        attention_mask = (input_ids != tokenizer.pad_token_id).to(dtype=torch.long) if tokenizer.pad_token_id is not None else torch.ones_like(input_ids)
        return self.model(input_ids=input_ids, attention_mask=attention_mask).logits

wrapped_model = WrappedModel(student_model)

# Export the non-quantized model to ONNX format
torch.onnx.export(wrapped_model, dummy_input["input_ids"], "model.onnx", export_params=True, input_names=['input_ids'], output_names=['output'], opset_version=14)

print("Export to ONNX complete!")

In [ ]:
pip install onnxruntime

In [ ]:
import onnx
import onnxruntime
import onnxruntime.quantization as quantization

# Load your ONNX model
model_path = "model.onnx"
onnx_model = onnx.load(model_path)

# Quantize the model using ONNX Runtime
quantized_model_path = "quantized_model.onnx"
quantization.quantize_dynamic(model_path, quantized_model_path)

print("Post-training quantization complete!")

In [ ]:
import torch
import torch.nn as nn
import torch.onnx
from transformers import AutoModelForCausalLM, AutoTokenizer

# Initialize student model and tokenizer
student_model = AutoModelForCausalLM.from_pretrained("distilgpt2")

# Create a dummy input for the export
tokenizer = AutoTokenizer.from_pretrained("distilgpt2")
dummy_input = tokenizer("This is a dummy input", return_tensors="pt")

# Create a wrapper model to handle input_ids only
class WrappedModel(nn.Module):
    def __init__(self, model):
        super(WrappedModel, self).__init__()
        self.model = model

    def forward(self, input_ids):
        attention_mask = (input_ids != tokenizer.pad_token_id).to(dtype=torch.long) if tokenizer.pad_token_id is not None else torch.ones_like(input_ids)
        return self.model(input_ids=input_ids, attention_mask=attention_mask).logits

wrapped_model = WrappedModel(student_model)

# Export the non-quantized model to ONNX format
torch.onnx.export(wrapped_model, dummy_input["input_ids"], "model.onnx", export_params=True, input_names=['input_ids'], output_names=['output'], opset_version=14)

print("Export to ONNX complete!")

import onnx
import onnxruntime
import onnxruntime.quantization as quantization

# Load your ONNX model
model_path = "model.onnx"
onnx_model = onnx.load(model_path)

# Quantize the model using ONNX Runtime
quantized_model_path = "quantized_model.onnx"
quantization.quantize_dynamic(model_path, quantized_model_path)

print("Post-training quantization complete!")